In [1]:
import torch
import os

import pandas as pd

In [9]:
def l0_eps(xpl, eps=1e-8):
    # returns ratio of points whose influence (relative to the maximum influence) is smaller than eps
    xpl_norm = xpl / xpl.max(dim=1).values.unsqueeze(1)
    return (xpl_norm.abs() < eps).float().sum(dim=1).mean().item() / xpl.shape[1]

In [10]:
def hoyer_measure(xpl):
    """
    Calculate the Hoyer measure of sparsity for each row in a tensor.
    
    The Hoyer measure is defined as:
        (sqrt(n) - L1/L2) / (sqrt(n) - 1)
    
    where n is the dimension, L1 is the L1 norm, and L2 is the L2 norm.
    The measure ranges from 0 (completely dense) to 1 (completely sparse).
    
    Args:
        tensor (torch.Tensor): Input tensor of shape [m, n]
        
    Returns:
        torch.Tensor: Tensor of shape [m] containing the Hoyer measure for each row
    """
    # Get the dimension of each row
    n = xpl.shape[1]
    
    # Calculate L1 norm for each row (sum of absolute values)
    l1_norm = torch.norm(xpl, p=1, dim=1)
    
    # Calculate L2 norm for each row (Euclidean norm)
    l2_norm = torch.norm(xpl, p=2, dim=1)
    
    # Handle zero vectors to avoid division by zero
    # When L2 norm is 0, the vector is all zeros, so sparsity should be 1
    mask = l2_norm == 0
    
    # Calculate Hoyer measure
    sqrt_n = torch.sqrt(torch.tensor(n, dtype=xpl.dtype))
    hoyer = torch.zeros_like(l1_norm)
    
    # Apply formula only for non-zero vectors
    non_zero_indices = ~mask
    hoyer[non_zero_indices] = (sqrt_n - l1_norm[non_zero_indices] / l2_norm[non_zero_indices]) / (sqrt_n - 1)
    
    # Set measure to 1 for all-zero vectors
    hoyer[mask] = 1.0
    
    return hoyer.mean().item()

In [11]:
def find_xpl_tensor(dir):
    if os.path.isdir(dir):
        for file in os.listdir(dir):
            if file.endswith("_all"):
                path = os.path.join(dir, file)
                return path
    return None

In [16]:
path = os.getcwd()

df_list = []
for ds in ['MNIST', 'CIFAR', 'AWA']:
    data = []
    for ds_type in ['std']:
        for xai_method in ["dualview",
                           #"feature_similarity_dot", "feature_similarity_cos", "feature_similarity_l2", 
                           "graddot", "gradcos",
                           #"input_similarity_dot", "input_similarity_cos", "input_similarity_l2"
                           "lissa", "representer", "tracin", "trak", 
        ]:                  
            if xai_method == "dualview":
                for C in [100., 10., 1., 1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6]:
                    xpl_dir=os.path.join(path, "explanations", ds, ds_type, xai_method + "_" + str(C))
                    xpl_path=find_xpl_tensor(xpl_dir)
                    if xpl_path is not None:
                        xpl=torch.load(xpl_path, map_location=torch.device('cpu'))

                        l0=l0_eps(xpl)
                        hoyer=hoyer_measure(xpl)
                        data.append({'xai_method': xai_method + "_" + str(C), 'l0_eps_norm': l0, 'hoyer_measure': hoyer})
            else:
                xpl_dir=os.path.join(path, "explanations", ds, ds_type, xai_method)
                xpl_path=find_xpl_tensor(xpl_dir)
                if xpl_path is not None:
                    xpl=torch.load(xpl_path, map_location=torch.device('cpu'))

                    l0=l0_eps(xpl)
                    hoyer=hoyer_measure(xpl)
                    data.append({'xai_method': xai_method, 'l0_eps_norm': l0, 'hoyer_measure': hoyer})
    df = pd.DataFrame(data)
    df.set_index('xai_method', inplace=True)
    df.style.set_caption(f"Sparsity of explanations for {ds} dataset")
    df_list.append((ds, df))
    print(ds)
    print(df)
    print('\n---------\n')

MNIST
              l0_eps_norm  hoyer_measure
xai_method                              
dualview_0.1     0.998622       0.979259

---------



KeyError: "None of ['xai_method'] are in the columns"

In [20]:
for df in df_list:
    print(df[0], "\n")
    print(df[1].to_latex())
    print("\n")

MNIST 

\begin{tabular}{lrr}
\toprule
 & l0_eps_norm & hoyer_measure \\
xai_method &  &  \\
\midrule
dualview_0.1 & 0.998622 & 0.979259 \\
\bottomrule
\end{tabular}



